# 通过Gradio快速展示功能

In [1]:
!pip install gradio

In [2]:
import os
from openai import AzureOpenAI

deployment = "gpt-4o"
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

def get_response(inputs):
    response = client.chat.completions.create(
        model=deployment,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": inputs}
        ],
        temperature=0.8,
        max_tokens=256,
    )
    return response.choices[0].message.content


In [3]:
# 不传入上下文，没有记忆
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

你好，很高兴听到你有健身的计划！作为一个人工智能助手，我没有实际的活动，但我随时准备好帮助你。你今天的健身计划是什么？如果你有任何关于健身的问题或者需要一些建议，我很乐意帮忙。
每个人的健身时间安排可能会有所不同，要了解你具体的健身时间安排，还需要更多信息。一般来说，很多人会选择在工作日的晚上或者周末进行健身，因为这些时间段较为空闲。不过，你的具体健身时间取决于你的个人日程安排和生活习惯。如果你有记录健身的习惯，可以查看你的健身记录来确定你一般周几健身。


In [4]:
import gradio as gr

def respond(message, chat_history):
    bot_message = get_response(message)
    chat_history.append((message, bot_message)) # 保存历史对话记录，用于显示
    return bot_message, chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240)
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button(value="提交")
    
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    
gr.close_all()
demo.launch()

Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


# 提供记忆

In [5]:
def get_response(inputs):
    response = client.chat.completions.create(
        model=deployment,
        messages=inputs,
        temperature=0.8,
        max_tokens=256,
    )
    return response.choices[0].message.content

def history_to_prompt(chat_history): # 将对话内容保存在一个List里
    msg = [{"role": "system", "content": "You are an AI assistant."}]
    i = 0
    for round_trip in chat_history: # 将List里的内容，组成 ChatCompletion的 messages部分，{role，content} dict
        msg.append({"role": "user", "content": round_trip[0]})
        msg.append({"role": "assistant", "content": round_trip[1]})
    return msg

def respond(message, chat_history):
    his_msg = history_to_prompt(chat_history) #并装历史会话，ChatCompletion的 messages部分格式
    his_msg.append({"role": "user", "content": message}) # 放入当前用户问题
    bot_message = get_response(his_msg)
    chat_history.append((message, bot_message)) # 将用户问题和返回保存到 历史记录 List
    return "", chat_history

In [6]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("提交")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=False, debug=True)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
